In [26]:
from google.cloud import aiplatform
from google.cloud import bigquery

import pandas as pd
import numpy as np
from datetime import date,datetime,timedelta

import math
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf  # constantly registered to load model 
print(tf.__version__)
print(tfdf.__version__)

2.12.0
1.4.0


In [27]:
# import functions_framework
# @functions_framework.http
# def xgb_tf_predict_incident_severity(request):

In [24]:
option=1

_model='model_binary_gb_tf'   # the is resonable because unsseen and tune 70&


model_gs_path=f"gs://demo-tuned-tf-incident-pongthorn/{_model}"
print(model_gs_path)

projectId="pongthorn"
dataset_id="SMartML"


if option==1:
    unusedCols_unseen=['id','severity_name','open_to_close_hour']
else:
    unusedCols_unseen=['id','severity_name','range_open_to_close_hour']
    

gs://demo-tuned-tf-incident-pongthorn/model_binary_gb_tf


# Load Model

In [25]:
abc_model = tf.keras.models.load_model(model_gs_path)  
print(abc_model.summary())
# abc_model = tf.keras.models.load_model(model_local_path) 

OSError: No file or directory found at gs://demo-tuned-tf-incident-pongthorn/model_binary_gb_tf

# Make Prediction

In [12]:
client = bigquery.Client(project=projectId)
new_data_table_id=f"{projectId}.{dataset_id}.new2_incident"

query_result=client.query(f"SELECT * FROM {new_data_table_id}")
df=query_result.to_dataframe()

unseen =df.drop(columns=unusedCols_unseen)
print(unseen.info())
unseen.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   severity_id               80 non-null     Int64 
 1   sla                       80 non-null     object
 2   product_type              80 non-null     object
 3   brand                     80 non-null     object
 4   service_type              80 non-null     object
 5   incident_type             80 non-null     object
 6   range_open_to_close_hour  80 non-null     object
dtypes: Int64(1), object(6)
memory usage: 4.6+ KB
None


,severity_id,sla,product_type,brand,service_type,incident_type,range_open_to_close_hour
70,4,24x7 4Hrs Resolution Time,Access Point,Cisco,Request,General Incident,soonest
71,4,24x7 4Hrs Resolution Time,Access Point,Cisco,Request,General Incident,soonest
72,4,24x7 4Hrs Resolution Time,Software,Veeam,Request,Software,soonest
73,4,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,soon
74,4,24x7 4Hrs Resolution Time,Server,Oracle,Request,OS / Firmware,soonest
75,4,24x7 4Hrs Resolution Time,Server,Oracle,Request,Software,soonest
76,4,24x7 4Hrs Resolution Time,Server,Oracle,Request,Software,soonest
77,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Software,soonest
78,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Software,soon
79,4,24x7 4Hrs Response Time,Storage,Veritas,Incident,General Incident,soon


In [13]:
unseen_ds= tfdf.keras.pd_dataframe_to_tf_dataset(unseen.drop(columns=['severity_id']))
print(unseen_ds)

<_PrefetchDataset element_spec={'sla': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'product_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'brand': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'service_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'incident_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'range_open_to_close_hour': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>


In [14]:
predResultList=abc_model.predict(unseen_ds)
predServerityIDList=[]
for predResult in predResultList:
    _class=tf.argmax(predResult,-1).numpy()
    pred_seveirty_id=map_severity_to_class[_class]
    predServerityIDList.append(pred_seveirty_id)
    print(f"{predResult} : {_class} as severity#{pred_seveirty_id}")
    
dfPred=pd.DataFrame(data=predServerityIDList,columns=["pred_severity_id"])   
df=pd.concat([dfPred,df],axis=1)
# df

1/1 [==============================] - 0s 102ms/step
[0.01513131 0.1000897  0.796147   0.08863197] : 2 as severity#2
[0.0154452  0.07310431 0.84545916 0.06599136] : 2 as severity#2
[0.0154452  0.07310431 0.84545916 0.06599136] : 2 as severity#2
[0.0154452  0.07310431 0.84545916 0.06599136] : 2 as severity#2
[0.0154452  0.07310431 0.84545916 0.06599136] : 2 as severity#2
[0.0154452  0.07310431 0.84545916 0.06599136] : 2 as severity#2
[0.0154452  0.07310431 0.84545916 0.06599136] : 2 as severity#2
[0.0154452  0.07310431 0.84545916 0.06599136] : 2 as severity#2
[0.01336976 0.04342543 0.93415433 0.00905051] : 2 as severity#2
[0.01513131 0.1000897  0.796147   0.08863197] : 2 as severity#2
[0.0154452  0.07310431 0.84545916 0.06599136] : 2 as severity#2
[0.00982261 0.04917946 0.9345066  0.0064913 ] : 2 as severity#2
[0.02062506 0.12977083 0.8195875  0.03001656] : 2 as severity#2
[0.01779265 0.08065245 0.8698259  0.03172899] : 2 as severity#2
[0.02177456 0.06510144 0.8986616  0.01446236] : 2 a

2023-07-12 04:37:51.892276: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [80]
	 [[{{node Placeholder/_2}}]]


# Evaluate model

In [15]:
from sklearn.metrics import confusion_matrix,classification_report
className=list(set().union(list(df['pred_severity_id'].unique()),list(df['severity_id'].unique())))
className
actualClass=[  f'actual-{x}' for x in  className]
predictedlClass=[  f'pred-{x}' for x in className]
y_true=list(df['severity_id'])
y_pred=list(df['pred_severity_id'])
cnf_matrix = confusion_matrix(y_true,y_pred)
cnf_matrix

# #index=actual , column=prediction
cm_df = pd.DataFrame(cnf_matrix,
                     index = actualClass, 
                     columns = predictedlClass)
print(cm_df)
print(classification_report(y_true, y_pred, labels=className))

          pred-1  pred-2  pred-3  pred-4
actual-1       0       0       0       0
actual-2       0      29       3       0
actual-3       1       4      13       1
actual-4       0       8       3      18
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.71      0.91      0.79        32
           3       0.68      0.68      0.68        19
           4       0.95      0.62      0.75        29

    accuracy                           0.75        80
   macro avg       0.58      0.55      0.56        80
weighted avg       0.79      0.75      0.75        80



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Write Prediction Result to BQ

In [16]:
df=df[['id','severity_id','pred_severity_id']]
df['prediction_datetime']=datetime.now()
print(df)

      id  severity_id  pred_severity_id        prediction_datetime
0   2766            2                 2 2023-07-12 04:37:59.068290
1   3107            2                 2 2023-07-12 04:37:59.068290
2   3105            2                 2 2023-07-12 04:37:59.068290
3   3104            2                 2 2023-07-12 04:37:59.068290
4   3103            2                 2 2023-07-12 04:37:59.068290
..   ...          ...               ...                        ...
75  3094            4                 2 2023-07-12 04:37:59.068290
76  3093            4                 2 2023-07-12 04:37:59.068290
77  3053            4                 4 2023-07-12 04:37:59.068290
78  3052            4                 4 2023-07-12 04:37:59.068290
79  3029            4                 2 2023-07-12 04:37:59.068290

[80 rows x 4 columns]


In [17]:
predictResult_table_id=f"{projectId}.{dataset_id}.new2_result_prediction_incident"

try:
    client.get_table(predictResult_table_id)  # Make an API request.
    print("Predict Result Table {} already exists.".format(predictResult_table_id))
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("severity_id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("pred_severity_id", "INTEGER", mode="REQUIRED"),       
    bigquery.SchemaField("prediction_datetime", "DATETIME", mode="REQUIRED") 
    ]

    table = bigquery.Table(predictResult_table_id,schema=schema)
    # table.time_partitioning = bigquery.TimePartitioning(
    # type_=bigquery.TimePartitioningType.DAY,field="prediction_item_date")
    
    table = client.create_table(table)  # Make an API request.
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Predict Result Table pongthorn.SMartML.new2_result_prediction_incident already exists.


In [18]:
def loadDataFrameToBQ():
    # WRITE_TRUNCATE , WRITE_APPEND
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_TRUNCATE",
        )

        job = client.load_table_from_dataframe(
            df, predictResult_table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total Prediction ML ", len(df), "Imported bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex

Total Prediction ML  80 Imported bigquery successfully


In [19]:
# return 'All incidents has been predicted completely.'